## Spatial Modeling and Analytics
## Try it 2 - Spatial Vector Data in R

Again, let's drop into the Notebook view and take a look at some R code to display and analyze vector data. 

Click the big X in the top left corner. then click on the Run button on each cell below. 

In [ ]:
## Import Packages
library(ggplot2)
library(leaflet.esri)
library(sf)

We begin by loading the boundaries and some sudden infant death syndrome data for the counties in North Carolina from an ArcGIS shapefile into an R spatial data frame we're calling *sids.nc.* 

In [ ]:
## Get data
sids.nc <- st_read(system.file("shape/nc.shp", package="sf"))

### Print the first 3 rows of the geometry dataset
head(sids.nc, 3)

In [ ]:
## Make a static plot of the data
plot(sids.nc)

Notice that this is dumbly plotting all the variables, even ones that don't make sense to plot this way like COUNTY_ID and NAME. 

Let's make a map that makes sense, the count of sundden infant deaths in 1979...

In [ ]:
### Make a static plot for one variable SID79
plot(sids.nc["SID79"], breaks = "jenks")

In [ ]:
## Plot New Born Counts Against SIDS as a Line Plot
ggplot(sids.nc, aes(x=NWBIR79, y=SID79))  +
  geom_point( size = 3) + geom_line(size = 1, alpha=0.4, color = 'red') +
  xlab("Number of New Borns") + ylab("Number of SIDS Incidents") +
  theme_light()

In [ ]:
## Make A Map with ggplot2
p <- ggplot() +
  geom_sf(data = sids.nc, aes(fill = SID79)) +
  scale_fill_gradientn(colours = hcl.colors(10, alpha = 0.5))
p

In [ ]:
## 5. Defining Subsets By Indexing
### Make Subsets by Row Indices
sids.nc.row.subset <- sids.nc[1:15,]
p2 <- ggplot() +
  geom_sf(data = sids.nc ) + 
  geom_sf(data = sids.nc.row.subset, aes(fill = SID79) )
p2


In [ ]:
## 6. Defining Subsets by Intersection with Bounding Box
bbox <- st_bbox(sids.nc)
print(bbox)

In [ ]:
### Define a New Bounding Box to Intersect
new_bbox <- st_bbox(c(xmin = -80,
              ymin = 35,
              xmax = -77, 
              ymax = 36), crs = st_crs(sids.nc))

### Convert the Bounding Box to A Feature
new_bbox <- st_as_sfc(new_bbox)

### Intersect Features
sids.subset.bbox <- st_intersection(sids.nc, new_bbox)

p2 <- ggplot() +
  geom_sf(data = sids.nc ) + 
  geom_sf(data = sids.subset.bbox, aes(fill = SID79) )
p2

In [ ]:
## Reproject Data to WGS84 (CRS 4326) and Make an Interactive Map
sids.nc.wgs84 <- st_transform(sids.nc, crs = 4326)

### Make a Leaflet map of the data
LeafMap <- leaflet(sids.nc.wgs84) %>%
  addEsriBasemapLayer(esriBasemapLayers$Imagery) %>%
  addPolygons(color = "#000000", weight = 1, opacity = 1.0, fillOpacity = 0.5,
              fillColor = ~colorQuantile("YlOrRd", SID79)(SID79),
              highlightOptions = highlightOptions(color = "white",
                                                  bringToFront = TRUE),
              popup = paste0("<b>SID Cases in 1979:</b> ", sids.nc.wgs84$SID79,
                             "<br>", "<b>New Births in 1979</b>: ", 
                             sids.nc.wgs84$NWBIR79))
LeafMap

Now we go back to some information about how you can use R with GIS.

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" 
href="sma-5.ipynb">Click here to go to the next notebook</a></font>